# 4. LangChain 기초


In [ ]:
#  코랩사용시
# import os
# from google.colab import userdata

# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

In [1]:
# 로컬 구동시
import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## 4.1. LangChain 개요


### LangChain 설치


In [ ]:
!pip install langchain-core==0.3.0 langchain-openai==0.2.0 pydantic==2.9.2

### LangSmith 설정


In [3]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
# 코랩사용시
# os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")
# 로컬 구동시
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 4.2. LLM / Chat model


### LLM


In [4]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("안녕하세요. ")
print(ai_message)



안녕하세요. 반가워요.


### Chat model


In [5]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요! 저는 이영범이라고 합니다!"),
    AIMessage(content="안녕하세요, 이영범님! 어떤 도움이 필요하신가요?"),
    HumanMessage(content="제 이름을 아시나요?"),
]

ai_message = model.invoke(messages)
print(ai_message.content)

네, 이영범님이라고 말씀하셨습니다. 어떻게 도와드릴까요?


### 스트리밍


In [6]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

messages = [
    SystemMessage("You are a helpful assistant."),
    HumanMessage("안녕하세요!"),
]

for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

안녕하세요! 어떻게 도와드릴까요?

## 4.3. Prompt template


### PromptTemplate


In [9]:
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""다음 요리의 레시피를 생각해 주세요.

요리명: {dish}""")

prompt_value = prompt.invoke({"dish": "곰탕"})
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 곰탕


#### ＜보충: 프롬프트 변수가 1개인 경우＞


In [10]:
prompt_value = prompt.invoke("불고기")
print(prompt_value.text)

다음 요리의 레시피를 생각해 주세요.

요리명: 불고기


### ChatPromptTemplate


In [11]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

prompt_value = prompt.invoke({"dish": "된장찌게"})
print(prompt_value)

messages=[SystemMessage(content='사용자가 입력한 요리의 레시피를 생각해 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='된장찌게', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder
대화 이력처럼 여러 메시지가 들어가는 langchain의 플레이스 홀더

In [ ]:
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("chat_history", optional=True),
        ("human", "{input}"),
    ]
)

prompt_value = prompt.invoke(
    {
        "chat_history": [
            HumanMessage(content="안녕하세요! 저는 이영범이라고 합니다!"),
            AIMessage("안녕하세요, 이영범님! 어떻게 도와드릴까요?"),
        ],
        "input": "제 이름을 아시나요?",
    }
)
print(prompt_value)

messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content='안녕하세요! 저는 존이라고 합니다!', additional_kwargs={}, response_metadata={}), AIMessage(content='안녕하세요, 존님! 어떻게 도와드릴까요?', additional_kwargs={}, response_metadata={}), HumanMessage(content='제 이름을 아시나요?', additional_kwargs={}, response_metadata={})]


### LangSmith의 Prompts


In [13]:
from langsmith import Client

# client = Client()
# prompt = client.pull_prompt("ychoi/recipe")

# prompt_value = prompt.invoke({"dish": "카레"})

client = Client(api_key=os.getenv("LANGCHAIN_API_KEY"))
# 당신은 뛰어난 해설가입니다. 전달되는 단어를 최소 100자, 최대 200자 이내로 설명해주세요. 지식을 확장할 수 있도록 중요한 개념이나 사실을 언급해 주세요.
prompt = client.pull_prompt("test", include_model=True)

prompt_value = prompt.invoke({"word": "젠슨황이 AI 시대에 기여한 점"})
print(prompt_value)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/json/decoder.py:337: UserWarning: WARNING! extra_headers is not default parameter.
                extra_headers was transferred to model_kwargs.
                Please confirm that extra_headers is what you intended.
  obj, end = self.raw_decode(s, idx=_w(s, 0).end())


content='엔비디아 창업자 겸 CEO 젠슨 황은 GPU·CUDA 생태계를 확장해 딥러닝 학습·추론을 가속하고 DGX·A100·cuDNN·TensorRT 등 하드웨어·소프트웨어 통합으로 연구와 산업의 AI 혁신을 촉진했다.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 1042, 'prompt_tokens': 69, 'total_tokens': 1111, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 960, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-bba330e7-c7da-4882-81ab-c931629ef8b9-0' usage_metadata={'input_tokens': 69, 'output_tokens': 1042, 'total_tokens': 1111}


### (칼럼) 멀티모달 모델의 입력 처리


In [17]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "이미지를 설명해 주세요."},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/ychoi-kr/langchain-book/main/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

이 이미지는 책 표지입니다. 제목은 "챗GPT와 랭체인을 활용한 LLM 기반 AI 앱 개발"입니다. 표지에는 벌 모양의 종이접기 도안이 그려져 있으며, 노란색과 파란색 줄무늬가 특징입니다. 책은 랭체인과 LLM 애플리케이션 구축에 관한 내용을 다루고 있는 것으로 보입니다.


## 4.4. Output parser


### PydanticOutputParser를 사용한 Python 객체 변환
LLM의 출력을 Python 객체로 변환 가능 -> 추후에 등장할 with_structured_output을 사용하는 것을 권장

In [ ]:
from pydantic import BaseModel, Field

# 레시피 모델 정의
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [15]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [16]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [17]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [19]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "카레"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

=== role: system ===
사용자가 입력한 요리의 레시피를 생각해 주세요.

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of the dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
=== role: user ===
카레


In [20]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

{
  "ingredients": [
    "닭고기 500g",
    "양파 1개",
    "감자 2개",
    "당근 1개",
    "카레 가루 3큰술",
    "코코넛 밀크 400ml",
    "식용유 2큰술",
    "소금 약간",
    "후추 약간",
    "물 2컵"
  ],
  "steps": [
    "양파를 잘게 썰고, 감자와 당근은 큐브 모양으로 자릅니다.",
    "큰 냄비에 식용유를 두르고 중불에서 양파를 볶아 투명해질 때까지 볶습니다.",
    "닭고기를 추가하고 겉면이 노릇해질 때까지 볶습니다.",
    "감자와 당근을 넣고 잘 섞은 후, 카레 가루를 넣고 볶습니다.",
    "물과 코코넛 밀크를 추가하고 끓입니다.",
    "끓기 시작하면 불을 줄이고 뚜껑을 덮고 20-30분간 끓입니다.",
    "소금과 후추로 간을 맞추고, 원하는 농도가 될 때까지 더 끓입니다.",
    "완성된 카레를 밥과 함께 서빙합니다."
  ]
}


In [24]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기 500g', '양파 1개', '감자 2개', '당근 1개', '카레 가루 3큰술', '식용유 2큰술', '소금 약간', '후추 약간', '물 4컵'] steps=['닭고기를 한 입 크기로 자르고, 소금과 후추로 간을 한다.', '양파는 다지고, 감자와 당근은 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 양파를 볶아 투명해질 때까지 볶는다.', '닭고기를 넣고 겉면이 노릇해질 때까지 볶는다.', '감자와 당근을 넣고 함께 볶는다.', '물 4컵을 붓고 끓인다.', '끓기 시작하면 불을 줄이고, 카레 가루를 넣고 잘 섞는다.', '약한 불에서 20분 정도 끓여서 재료가 부드러워질 때까지 조리한다.', '완성된 카레를 밥과 함께 서빙한다.']


### StrOutputParser
LLM의 출력을 텍스트로 변환하는데 사용. LCEL의 구성 요소로서 중요한 역할을 함

In [21]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="안녕하세요. 저는 AI 어시스턴트입니다.")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
안녕하세요. 저는 AI 어시스턴트입니다.


## 4.5. Chain—LangChain Expression Language(LCEL) 개요
LLM 애플리케이션에서는 단순히 LLM에 입력하고 출력을 얻는 것으로 끝나지 않고, 처리를 연쇄적으로 연결(chaining)하고 싶은 경우가 많다

### prompt와 model 연결


In [22]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [23]:
chain = prompt | model

In [24]:
ai_message = chain.invoke({"dish": "카레"})
print(ai_message.content)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 돼지고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 피망, 버섯, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣어 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

5. **물 붓기**: 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 냉장고에 보관하면 맛이 더 깊어지므로, 다음 날 먹는 것도 추천합니다.
- 다양한 채소나 해산물을 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### StrOutputParser를 연결에 추가


In [ ]:
from langchain_core.output_parsers import StrOutputParser

# Chat model의 출력인 AIMessage를 텍스트로 변환
chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "카레"})
print(output)

카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 돼지고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 코코넛 밀크 또는 물 2컵
- 식용유 2 큰술
- 소금, 후추 약간
- 선택 재료: 마늘, 생강, 피망, 버섯 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다. 선택 재료도 적당한 크기로 썰어줍니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣고 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다. 이때 마늘과 생강을 추가하면 향이 더해집니다.

5. **카레 가루 추가**: 카레 가루를 넣고 잘 섞어줍니다. 이때 향신료의 향이 올라옵니다.

6. **액체 추가**: 코코넛 밀크 또는 물을 부어주고, 소금과 후추로 간을 맞춥니다. 끓어오르면 불을 줄이고 뚜껑을 덮고 20-30분 정도 끓입니다. 중간에 저어주면 좋습니다.

7. **완성**: 채소가 부드러워지고 소스가 걸쭉해지면 불을 끄고, 필요에 따라 추가 간을 맞춥니다.

8. **서빙**: 밥과 함께 따뜻하게 서빙합니다. 원한다면 고수나 파슬리로 장식해도 좋습니다.

맛있게 드세요!


### PydanticOutputParser를 사용한 연결


In [ ]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# 클래스를 정의하고
class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

# output_parser를 준비
output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요.\n\n{format_instructions}"),
        ("human", "{dish}"),
    ]
)

# prompt.partial: 프로프트 템플릿에서 일부 변수를 미리 채워두는 함수
prompt_with_format_instructions = prompt.partial(
    format_instructions=output_parser.get_format_instructions()
)

model = ChatOpenAI(model="gpt-4o-mini", temperature=0).bind(
    response_format={"type": "json_object"}
)

In [27]:
chain = prompt_with_format_instructions | model | output_parser

In [26]:
recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class 'str'>
카레는 다양한 재료와 향신료를 사용하여 만드는 맛있는 요리입니다. 아래는 기본적인 카레 레시피입니다.

### 재료
- 고기 (닭고기, 소고기, 양고기 등) 300g
- 양파 1개
- 감자 1개
- 당근 1개
- 카레 가루 2-3 큰술
- 식용유 2 큰술
- 물 3컵
- 소금, 후추 약간
- 선택 재료: 피망, 버섯, 완두콩 등

### 조리 방법
1. **재료 손질**: 고기는 한 입 크기로 자르고, 양파는 다지고, 감자와 당근은 깍둑썰기 합니다.

2. **양파 볶기**: 큰 냄비에 식용유를 두르고 중불에서 다진 양파를 넣어 투명해질 때까지 볶습니다.

3. **고기 추가**: 양파가 볶아지면 고기를 넣고 겉면이 익을 때까지 볶습니다.

4. **채소 추가**: 감자와 당근을 넣고 함께 볶아줍니다.

5. **물 붓기**: 재료가 잘 섞이면 물을 붓고 끓입니다. 끓기 시작하면 불을 줄이고 중약불로 15-20분 정도 끓입니다.

6. **카레 가루 추가**: 카레 가루를 넣고 잘 섞은 후, 다시 10분 정도 끓입니다. 필요에 따라 소금과 후추로 간을 맞춥니다.

7. **완성**: 카레가 걸쭉해지면 불을 끄고, 밥과 함께 서빙합니다.

### 팁
- 카레는 냉장고에 보관하면 맛이 더 깊어지므로, 다음 날 먹는 것도 추천합니다.
- 다양한 채소나 해산물을 추가하여 나만의 카레를 만들어 보세요!

맛있게 드세요!


### （칼럼）with_structured_output
실제로 LangChain에서 LLM에 구조화된 데이터를 출력하게 할 때는 보다 간단한 with_structured_output을 사용. 모든 Chat model에서 사용할 수 있는 것은 아니며, ChatOpenAI와 같은 일부 model에서 사용 가능. with_structured_output은 내부적으로 Function calling을 사용하여 JSON 형식의 데이터를 출력

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "사용자가 입력한 요리의 레시피를 생각해 주세요."),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "카레"})
print(type(recipe))
print(recipe)

<class '__main__.Recipe'>
ingredients=['닭고기', '양파', '감자', '당근', '카레 가루', '코코넛 밀크', '식용유', '소금', '후추', '물'] steps=['닭고기를 한 입 크기로 썰고, 소금과 후추로 간을 한다.', '양파를 다지고, 감자와 당근을 깍둑썰기로 준비한다.', '팬에 식용유를 두르고 다진 양파를 넣어 볶다가 투명해질 때까지 볶는다.', '닭고기를 넣고 겉면이 익을 때까지 볶는다.', '감자와 당근을 추가하고 함께 볶는다.', '카레 가루를 넣고 잘 섞은 후, 물과 코코넛 밀크를 붓고 끓인다.', '중불로 20-30분간 끓여서 재료들이 부드러워질 때까지 조리한다.', '소금으로 간을 맞추고, 원하는 농도로 졸여서 완성한다.']


## 4.6.LangChain의 RAG 관련 컴포넌트
RAG(Retrieval-Augmented Generation: 끌어온 것으로 보강하여 생성)
LangChain의 공식 문서를 읽어 들여 gpt-4o-mioni에게 질문하는 예로 실제 흐름을 실행

### Document loader
https://python.langchain.com/docs/integrations/document_loaders/

In [29]:
!pip install langchain-community==0.3.0 GitPython==3.1.43

In [49]:
from langchain_community.document_loaders import GitLoader

# Document loader의 한 종류인 GitLoader를 사용해 .mdx라는 확장자의 파일을 읽어 들임
def file_filter(file_path: str) -> bool:
    return file_path.endswith(".mdx")


loader = GitLoader(
    clone_url="https://github.com/langchain-ai/docs",
    repo_path="./langchain",
    branch="main",
    file_filter=file_filter,
)

raw_docs = loader.load()
print(len(raw_docs))

2786


### Document transformer
문서를 어느 정도 길이의 청크로 분할하는 등의 문서를 변환하는 역할

In [34]:
!pip install langchain-text-splitters==0.3.0

In [50]:
from langchain_text_splitters import CharacterTextSplitter

# 문서를 청크로 분할
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

docs = text_splitter.split_documents(raw_docs)
print(len(docs))

Created a chunk of size 1772, which is longer than the specified 1000
Created a chunk of size 3159, which is longer than the specified 1000
Created a chunk of size 5522, which is longer than the specified 1000
Created a chunk of size 17087, which is longer than the specified 1000
Created a chunk of size 2380, which is longer than the specified 1000
Created a chunk of size 2113, which is longer than the specified 1000
Created a chunk of size 1598, which is longer than the specified 1000
Created a chunk of size 1217, which is longer than the specified 1000
Created a chunk of size 3182, which is longer than the specified 1000
Created a chunk of size 3092, which is longer than the specified 1000
Created a chunk of size 1441, which is longer than the specified 1000
Created a chunk of size 1456, which is longer than the specified 1000
Created a chunk of size 1092, which is longer than the specified 1000
Created a chunk of size 3002, which is longer than the specified 1000
Created a chunk of 

14941


### Embedding model
텍스트를 벡터화. OpenAIEmbeddings처럼 텍스트 벡터화에 사용할 수 있는 것이 Embedding model

In [51]:
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [ ]:
# 벡터화 예시
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

vector = embeddings.embed_query(query)
# 임베딩 벡터의 차원 
print(len(vector))
print(vector)

1536
[0.0063845072872936726, 0.008431448601186275, 0.023545239120721817, -0.027140924707055092, 0.03550197556614876, 0.006292448844760656, -0.0015609280671924353, 0.0173394326120615, 0.011491029523313046, -0.025689654052257538, -0.0010004560463130474, -0.002333946293219924, -0.009449503384530544, -0.008713037706911564, -0.015205848962068558, 0.04873669520020485, 0.0074783749878406525, -0.015043393708765507, -0.029198696836829185, 0.0004998896038159728, 0.025364741683006287, 0.033032648265361786, -0.026447780430316925, 0.05055619776248932, 0.025776296854019165, -0.04739372804760933, -0.017512720078229904, 0.034722186625003815, -0.013104756362736225, -0.10700412094593048, -0.01300728227943182, -0.05449845641851425, -0.025386402383446693, 0.03275106102228165, -0.015292491763830185, 0.05250566825270653, 0.04566086828708649, 0.009666111320257187, -0.011491029523313046, -0.01996038481593132, 0.012292477302253246, -0.004088467452675104, 0.015292491763830185, 0.004256337881088257, 0.0301517695

### Vector store
저장소인 Vector store에 문서를 벡터화하여 저장

In [39]:
!pip install langchain-chroma==0.1.4

In [ ]:
# 필터링된 문서로 처리
import os
os.environ["ANONYMIZED_TELEMETRY"] = "False"

from langchain_chroma import Chroma
from tqdm import tqdm
import time

# chroma_db 폴더가 이미 존재하면 기존 DB 로드, 없으면 새로 생성
db_path = "./chroma_db"

if os.path.exists(db_path):
    print(f"✓ 기존 DB를 로드합니다: {db_path}")
    db = Chroma(persist_directory=db_path, embedding_function=embeddings)
    print(f"총 {db._collection.count()} 개의 문서가 저장되어 있습니다.")
else:
    print("✗ DB가 없습니다. 새로 생성합니다...")
    
    # 먼저 문서 길이 확인
    doc_lengths = [len(doc.page_content) for doc in docs]
    print(f"평균 문서 길이: {sum(doc_lengths)/len(doc_lengths):.0f} 문자")
    print(f"최대 문서 길이: {max(doc_lengths)} 문자")
    print(f"총 문서 수: {len(docs)}")

    # 너무 긴 문서 제외 (예: 50,000자 이상)
    docs_filtered = [doc for doc in docs if len(doc.page_content) < 50000]
    print(f"필터링 후: {len(docs_filtered)} 문서")

    batch_size = 20
    db = None

    for i in tqdm(range(0, len(docs_filtered), batch_size)):
        batch = docs_filtered[i:i + batch_size]
        try:
            if db is None:
                db = Chroma.from_documents(batch, embeddings, persist_directory=db_path)
            else:
                db.add_documents(batch)
            time.sleep(0.5)
        except Exception as e:
            print(f"배치 {i} 실패: {e}")
    
    print(f"✓ DB 생성 완료: {db._collection.count()} 개의 문서 저장됨")

In [56]:
# Vector store 인스턴스에서 Retriever를 생성
retriever = db.as_retriever()

In [ ]:
query = "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?"

# 질문과 가까운, 즉 관련성이 높은 문서를 검색
context_docs = retriever.invoke(query)
print(f"len = {len(context_docs)}")

print(context_docs)
first_doc = context_docs[0]
print(f"metadata = {first_doc.metadata}")
print(first_doc.page_content)

Failed to send telemetry event CollectionQueryEvent: capture() takes 1 positional argument but 3 were given


len = 4
metadata = {'file_name': 'aws_s3_file.mdx', 'file_path': 'src/oss/python/integrations/document_loaders/aws_s3_file.mdx', 'file_type': '.mdx', 'source': 'src/oss/python/integrations/document_loaders/aws_s3_file.mdx'}
---
title: AWS S3 File
---

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html) is an object storage service.

>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

This covers how to load document objects from an `AWS S3 File` object.

```python
from langchain_community.document_loaders import S3FileLoader
```

```python
%pip install -qU  boto3
```

```python
loader = S3FileLoader("testing-hwc", "fake.docx")
```

```python
loader.load()
```

```output
[Document(page_content='Lorem ipsum dolor sit amet.', lookup_str='', metadata={'source': 's3://testing-hwc/fake.docx'}, lookup_index=0)]
```

## Configuring the AWS Boto3 client


### LCEL을 사용한 RAG Chain 구현
검색결과를 PromptTemplate에 context로 포함하여 LLM에 질문하고 답변을 받기

In [58]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_template('''\
다음 문맥만을 바탕으로 질문에 답변해 주세요.

문맥: """
{context}
"""

질문: {question}
''')

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [59]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

output = chain.invoke(query)
print(output)

네, AWS S3에서 데이터를 읽어 들이기 위한 Document loader가 있습니다. Python에서는 `S3FileLoader`를 사용하여 S3 파일 객체에서 문서 객체를 로드할 수 있습니다. JavaScript에서는 `S3Loader`를 사용하여 S3 파일 객체에서 문서 객체를 로드할 수 있습니다. 두 가지 모두 S3에서 파일을 로드하는 방법을 제공합니다.


### 임베딩 시각화
벡터 DB에 저장된 문서들의 임베딩을 2차원으로 축소하여 시각화합니다.

In [76]:
!pip install plotly==5.22.0 pandas==2.1.4 umap-learn==0.5.7 'nbformat>=4.2.0'

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import umap
import json

# 1. DB에서 모든 임베딩 가져오기
collection = db._collection
results = collection.get(include=['embeddings', 'documents', 'metadatas'])

embeddings_array = np.array(results['embeddings'])
documents = results['documents']

print(f"임베딩 개수: {len(embeddings_array)}")
print(f"임베딩 차원: {embeddings_array.shape[1]}")

# 2. UMAP으로 차원 축소 (고차원 -> 2D)
print("차원 축소 중...")
reducer = umap.UMAP(n_components=2, random_state=42, n_neighbors=15, min_dist=0.1)
embeddings_2d = reducer.fit_transform(embeddings_array)

print("차원 축소 완료!")

# 3. 모든 데이터를 Python 기본 타입의 리스트로 변환 (numpy 완전 제거)
x_coords = embeddings_2d[:, 0].tolist()
y_coords = embeddings_2d[:, 1].tolist()
texts = [str(doc[:200] + "..." if len(doc) > 200 else doc) for doc in documents]
sources = [str(meta.get('source', 'unknown').split('/')[-1]) for meta in results['metadatas']]

# 4. 색상 매핑 생성
unique_sources = sorted(list(set(sources)))
color_map = {src: idx for idx, src in enumerate(unique_sources)}
colors = [color_map[src] for src in sources]

print(f"데이터 타입 확인:")
print(f"x_coords type: {type(x_coords)}, element type: {type(x_coords[0])}")
print(f"y_coords type: {type(y_coords)}, element type: {type(y_coords[0])}")
print(f"colors type: {type(colors)}, element type: {type(colors[0])}")

# 5. plotly graph_objects로 직접 생성
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=x_coords,
    y=y_coords,
    mode='markers',
    marker=dict(
        size=5,
        color=colors,
        colorscale='Viridis',
        showscale=True,
        opacity=0.6
    ),
    text=texts,
    hovertemplate='<b>%{text}</b><extra></extra>'
))

fig.update_layout(
    title='LangChain Docs - 2D Document Embeddings',
    width=1200,
    height=800,
    xaxis_title='UMAP 1',
    yaxis_title='UMAP 2',
    hovermode='closest'
)

# 디버깅: figure를 dict로 변환해서 numpy 배열이 있는지 확인
try:
    fig_dict = fig.to_dict()
    # JSON 직렬화 테스트
    json.dumps(fig_dict)
    print("✓ Figure is JSON serializable")
except TypeError as e:
    print(f"✗ Figure contains non-serializable data: {e}")

fig.show()

print(f"\n2D 시각화 완료! 총 {len(x_coords)} 개의 문서가 표시되었습니다.")

#### 3D 시각화
마우스로 회전하며 3차원 공간에서 문서 클러스터를 탐색할 수 있습니다.

In [ ]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
import umap

# 1. DB에서 모든 임베딩 가져오기 (이미 있다면 재사용)
if 'embeddings_array' not in locals():
    collection = db._collection
    results = collection.get(include=['embeddings', 'documents', 'metadatas'])
    embeddings_array = np.array(results['embeddings'])
    documents = results['documents']

print(f"임베딩 개수: {len(embeddings_array)}")
print(f"임베딩 차원: {embeddings_array.shape[1]}")

# 2. UMAP으로 3D 차원 축소
print("3D 차원 축소 중...")
reducer_3d = umap.UMAP(n_components=3, random_state=42, n_neighbors=15, min_dist=0.1)
embeddings_3d = reducer_3d.fit_transform(embeddings_array)

print("3D 차원 축소 완료!")

# 3. DataFrame 생성 (모든 데이터를 Python 기본 타입으로 변환)
df_3d = pd.DataFrame({
    'x': [float(x) for x in embeddings_3d[:, 0]],
    'y': [float(y) for y in embeddings_3d[:, 1]],
    'z': [float(z) for z in embeddings_3d[:, 2]],
    'text': [str(doc[:200] + "..." if len(doc) > 200 else doc) for doc in documents],
    'source': [str(meta.get('source', 'unknown').split('/')[-1]) for meta in results['metadatas']]
})

# 4. 3D 인터랙티브 플롯
fig = go.Figure(data=[go.Scatter3d(
    x=df_3d['x'].tolist(),
    y=df_3d['y'].tolist(),
    z=df_3d['z'].tolist(),
    mode='markers',
    marker=dict(
        size=3,
        color=df_3d['source'].astype('category').cat.codes.tolist(),
        colorscale='Viridis',
        opacity=0.6,
        line=dict(width=0)
    ),
    text=df_3d['text'].tolist(),
    hovertemplate='<b>%{text}</b><br>x: %{x:.2f}<br>y: %{y:.2f}<br>z: %{z:.2f}<extra></extra>',
    name='Documents'
)])

fig.update_layout(
    title='LangChain Docs - 3D Document Embeddings (마우스로 회전 가능)',
    scene=dict(
        xaxis_title='UMAP 1',
        yaxis_title='UMAP 2',
        zaxis_title='UMAP 3',
        camera=dict(
            eye=dict(x=1.5, y=1.5, z=1.5)
        )
    ),
    width=1200,
    height=800,
    hovermode='closest'
)

fig.show()

print(f"\n3D 시각화 완료! 총 {len(df_3d)} 개의 문서가 표시되었습니다.")
print("💡 팁: 마우스로 드래그하여 회전, 스크롤로 확대/축소할 수 있습니다!")

### TensorFlow Projector 데이터 내보내기
질문의 임베딩을 함께 포함시켜 TensorFlow Projector(https://projector.tensorflow.org/) 에서 질문과 가까운 문서들을 시각적으로 찾을 수 있습니다.

In [ ]:
import csv
import os
import shutil

# 출력 디렉토리 재생성 (기존 파일 삭제)
output_dir = "./projector_data"
if os.path.exists(output_dir):
    shutil.rmtree(output_dir)
    print(f"✓ 기존 디렉토리 삭제: {output_dir}")
os.makedirs(output_dir, exist_ok=True)
print(f"✓ 새 디렉토리 생성: {output_dir}")

# 1. DB에서 임베딩 데이터 가져오기
collection = db._collection
results = collection.get(include=['embeddings', 'documents', 'metadatas'])

embeddings_array = results['embeddings']
documents = results['documents']
metadatas = results['metadatas']

print(f"\n문서 임베딩 개수: {len(embeddings_array)}")
print(f"임베딩 차원: {len(embeddings_array[0])}")

# 2. 질문들의 임베딩 생성
queries = [
    "AWS의 S3에서 데이터를 읽어 들이기 위한 Document loader가 있나요?",
    "LangChain에서 PDF 파일을 로드하는 방법",
    "Vector store는 어떻게 사용하나요?",
]

print(f"\n질문 임베딩 생성 중...")
query_embeddings = [embeddings.embed_query(q) for q in queries]
print(f"질문 임베딩 개수: {len(query_embeddings)}")

# 3. 문서 임베딩 + 질문 임베딩을 합쳐서 vectors.tsv로 저장
vectors_path = os.path.join(output_dir, "vectors.tsv")
with open(vectors_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t')
    
    # 문서 임베딩 작성
    for embedding in embeddings_array:
        writer.writerow(embedding)
    
    # 질문 임베딩 작성
    for embedding in query_embeddings:
        writer.writerow(embedding)

print(f"\n✓ 벡터 데이터 저장: {vectors_path}")
print(f"  파일 크기: {os.path.getsize(vectors_path) / 1024 / 1024:.1f} MB")

# 4. 메타데이터를 TSV로 저장 (metadata.tsv)
metadata_path = os.path.join(output_dir, "metadata.tsv")
with open(metadata_path, 'w', newline='', encoding='utf-8') as f:
    writer = csv.writer(f, delimiter='\t')
    
    # 헤더 작성 - Label과 Content만!
    writer.writerow(['Label', 'Content'])
    
    # 문서 메타데이터 작성
    for doc, meta in zip(documents, metadatas):
        preview = doc[:150].replace('\n', ' ').replace('\t', ' ')
        writer.writerow(['Document', preview])
    
    # 질문 메타데이터 작성
    for i, query in enumerate(queries, 1):
        query_text = query.replace('\n', ' ').replace('\t', ' ')
        writer.writerow([f'🔍QUERY_{i}', query_text])

print(f"✓ 메타데이터 저장: {metadata_path}")
print(f"  파일 크기: {os.path.getsize(metadata_path) / 1024:.1f} KB")

# 5. 검증: 메타데이터 파일 미리보기
print(f"\n✅ 메타데이터 파일 미리보기 (처음 5줄):")
with open(metadata_path, 'r', encoding='utf-8') as f:
    for i, line in enumerate(f):
        if i >= 5:
            break
        print(f"  {line.rstrip()}")

# 6. 통계 출력
print(f"\n📊 생성 완료!")
print(f"   - 문서: {len(documents)} 개 (Label: 'Document')")
print(f"   - 질문: {len(queries)} 개 (Label: '🔍QUERY_1', '🔍QUERY_2', '🔍QUERY_3')")
print(f"   - 총 벡터: {len(embeddings_array) + len(query_embeddings)} 개")

print(f"\n📊 TensorFlow Projector 사용 방법:")
print(f"1. https://projector.tensorflow.org/ 접속")
print(f"2. 좌측 상단 'Load' 버튼 클릭")
print(f"3. 파일 업로드:")
print(f"   - vectors.tsv: {vectors_path}")
print(f"   - metadata.tsv: {metadata_path}")
print(f"4. 'Color by' → 'Label' 선택")
print(f"5. 'T-SNE' 또는 'UMAP' 실행 (Iterations: 1000)")
print(f"6. 🔍QUERY 포인트 근처가 관련 문서")
print(f"\n💡 팁: 검색창에 '🔍' 입력하면 질문만 하이라이트")